# This notebook uses a 2D CNN to make predictions of apnea on the spectrograms for 60 seconds segments

Install libraries

In [ ]:
!pip install tqdm #install tqdm library on colab
!pip install torchmetrics
!pip install seaborn
!pip install scipy
!pip install sns
!pip install PIL

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 33.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sns: filename=sns-0.1-py3-none-any.whl size=2662 sha256=17ed0274fe5e8645aa0052506c6323a61b814a4c4d694375068ef2f2f6b20f0b
  Stored in directory: /root/.cache/pip/wheels/94/30/68/c7bbc5257894097a14adf5c7d6e7de56127c9936f27afe2416
Successfully built sns
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement PIL (from vers

In [ ]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize , LabelEncoder
import torch
import os
import torch.nn as nn
from torch import optim 
import torch.nn.functional as F
from PIL import Image
import numpy as np
#from scipy import signal
#from scipy.fft import fftshift
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from skimage import io
from skimage.color import rgba2rgb
import torchvision
from torch.utils.data import Dataset,DataLoader,TensorDataset
import torch.utils.data as data_utils
import torchvision.transforms as transforms  
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt 


True

Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Set up device

In [ ]:
#set torch to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #set gpu for device
torch.cuda.is_available()

Extract the images locally

In [ ]:
!unzip "/content/drive/My Drive/ML/Spectrogram_Hann.zip" -d "/content"

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: /content/Spectrogram_Hann/Train/spc_a02_422.png  
  inflating: /content/Spectrogram_Hann/Train/spc_a02_344.png  
  inflating: /content/Spectrogram_Hann/Train/spc_a01_287.png  
  inflating: /content/Spectrogram_Hann/Train/spc_a03_157.png  
  inflating: /content/Spectrogram_Hann/Train/spc_c06_119.png  
  inflating: /content/Spectrogram_Hann/Train/spc_b03_407.png  
  inflating: /content/Spectrogram_Hann/Train/spc_c02_57.png  
  inflating: /content/Spectrogram_Hann/Train/spc_a12_436.png  
  inflating: /content/Spectrogram_Hann/Train/spc_a02_10.png  
  inflating: /content/Spectrogram_Hann/Train/spc_a14_329.png  
  inflating: /content/Spectrogram_Hann/Train/spc_a08_119.png  
  inflating: /content/Spectrogram_Hann/Train/spc_c01_415.png  
  inflating: /content/Spectrogram_Hann/Train/spc_a16_65.png  
  inflating: /content/Spectrogram_Hann/Train/spc_c04_222.png  
  inflating: /content/Spectrogram_Hann/Trai

This cell defines useful constants

Change the path_c variable if google colab cannot be used

In [ ]:
path_c = '/content/'

out_directory_train = path_c + 'Spectrogram_Hann/Train/'
out_directory_test = path_c + 'Spectrogram_Hann/Test/'

# **pytorch model**

## DataSet class

The Dataset class, it has a dataframe as attribute to store a mapping annotations and png filenames. We also encode the annotations to binary.

To get one item, one retrieves a random image filename from the dataframe,
retrieves the actual file convert it to RGB, apply transformations and then we return this image with corresponding annotation.

In [ ]:
class SpectrogramDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(
            csv_file
        )  # load data input and output from csv file
        self.annotations.columns = ["name", "label"]
        #encode annotation
        self.annotations["label"] = self.annotations["label"].apply(
            lambda x: 1 if x == "A" else 0
        )
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)  # return length of csv file

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = Image.open(img_path).convert("RGB")
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)

        


## Hyperparameters


In [ ]:
in_channels = 3
num_classes = 2
learning_rate = 0.0001
batch_size = 32
num_epochs = 5

# Create loaders

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),])

train_set = SpectrogramDataset(csv_file = out_directory_train + 'spc_train_labels.csv', root_dir = out_directory_train, transform = transform)
test_set = SpectrogramDataset(csv_file = out_directory_test + 'spc_test_labels.csv', root_dir = out_directory_test, transform = transform)
 
train_loader = DataLoader(dataset= train_set, batch_size = batch_size, shuffle =True)
test_loader = DataLoader(dataset= test_set, batch_size = batch_size, shuffle =True)

# **Define the model class**

In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=1):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels, out_channels=128, kernel_size=(3, 3), stride=1
        )
        # convolution to extract features from data
        self.pool1 = nn.MaxPool2d(2, stride=2)

        self.conv2 = nn.Conv2d(
            in_channels=128, out_channels=64, kernel_size=(5, 5), stride=1
        )
        self.pool2 = nn.MaxPool2d(2, stride=2)

        self.conv3 = nn.Conv2d(
            in_channels=64, out_channels=32, kernel_size=(7, 7), stride=1
        )
        self.pool3 = nn.MaxPool2d(2, stride=2)

        # neuronal network training of data
        self.drop = nn.Dropout(p=0.6)
        self.fc1 = nn.Linear(26048, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))

        x = self.pool1(x)

        x = F.relu(self.conv2(x))

        x = self.pool2(x)

        x = F.relu(self.conv3(x))
        x = self.pool3(x)

        x = x.reshape(x.shape[0], -1)

        x = self.drop(x)

        x = torch.sigmoid(self.fc1(x))

        return x

Define model and add to device

In [ ]:
model = CNN(in_channels=in_channels, num_classes=1).to(device)
model = model.to(device = device)

Define loss and optimizers

In [ ]:
criterion =  nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Train Network and calculate accuracy and loss

In [ ]:
losses = []
accur = []
losses_test = []
accur_test = []

for epoch in range(num_epochs):
  i = 0 
  running_loss =0.0
  for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    # Get data to cuda if possible
    data = data.to(device=device)
    targets = targets.to(device=device)
    data = data.float()
    # forward
    scores = model(data)
    scores = torch.flatten(scores)
    scores = scores.to(torch.float)
    targets = targets.to(torch.float)
    loss = criterion(scores, targets)

    optimizer.zero_grad()
    loss.backward()

    # gradient descent or adam step
    optimizer.step()
    running_loss += loss.item()
    i+=1
    if i % 10 == 9:    # print every 4 mini-batches
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
        running_loss = 0.0




  2%|▏         | 9/428 [00:11<04:22,  1.60it/s]

[1,    10] loss: 0.614


  4%|▍         | 19/428 [00:16<03:23,  2.01it/s]

[1,    20] loss: 0.671


  7%|▋         | 29/428 [00:21<03:18,  2.01it/s]

...

[5,   410] loss: 0.319


 98%|█████████▊| 419/428 [03:33<00:04,  1.96it/s]

[5,   420] loss: 0.410


100%|██████████| 428/428 [03:37<00:00,  1.96it/s]


'\n  losses.append(loss.detach().numpy())\n  accur.append(acc)\n  losses_test.append(loss_test.detach().numpy())\n  accur_test.append(acc_test)\n  print("\n epoch {}\tloss_train : {}\t accuracy_train : {}%".format(epoch,loss,acc*100))\n  print("epoch {}\tloss_test : {}\t accuracy_test : {}%\n".format(epoch,loss_test,acc_test*100))\n  \n'

# Testing

This function returns the true_positives, false_positives, true_negatives, false_negatives

In [ ]:
def confusion(prediction, truth):
    """ Returns the confusion matrix for the values in the `prediction` and `truth`
    tensors, i.e. the amount of positions where the values of `prediction`
    and `truth` are
    - 1 and 1 (True Positive)
    - 1 and 0 (False Positive)
    - 0 and 0 (True Negative)
    - 0 and 1 (False Negative)
    """

    confusion_vector = prediction / truth
    # Element-wise division of the 2 tensors returns a new tensor which holds a
    # unique value for each case:
    #   1     where prediction and truth are 1 (True Positive)
    #   inf   where prediction is 1 and truth is 0 (False Positive)
    #   nan   where prediction and truth are 0 (True Negative)
    #   0     where prediction is 0 and truth is 1 (False Negative)

    true_positives = torch.sum(confusion_vector == 1).item()
    false_positives = torch.sum(confusion_vector == float('inf')).item()
    true_negatives = torch.sum(torch.isnan(confusion_vector)).item()
    false_negatives = torch.sum(confusion_vector == 0).item()

    return true_positives, false_positives, true_negatives, false_negatives

This function iterates through the test_loader, and calculates the different useful metrics: sensitivity, specificity, F1, Youden's index and accuracy.

In [ ]:
def check_accuracy(loader, model):
    model.eval()
    accuracy = 0.0
    total = 0.0
    if loader == "train":
        print("Checking accuracy on train data")
        with torch.no_grad():
            for data in train_loader:
                segments, labels = data
                # run the model on the test set to predict labels
                segments = segments.float()

                outputs = model(segments)

                ##print("labels: " ,   labels)

                # the label with the highest energy will be our prediction
                _, predicted = torch.max(outputs.data, 1)

                # print("predicted: ", predicted )

                TP = (predicted == labels == 1).sum()
                total += labels.size(0)
                accuracy += (predicted == labels).sum().item()

                # compute the accuracy over all test segments
            accuracy = 100 * accuracy / total
            print(f"Got {accuracy} ")

    else:
        print("Checking accuracy on test data")
        with torch.no_grad():
            true_positives, false_positives, true_negatives, false_negatives = (
                0,
                0,
                0,
                0,
            )
            predicted_tot = torch.zeros(0)
            labels_tot = torch.zeros(0)

            for data in test_loader:
                segments, labels = data
                # run the model on the test set to predict labels
                segments = segments.float()
                segments = segments.to(device=device)
                labels = labels.to(device=device)

                outputs = model(segments)

                # the label with the highest energy will be our prediction
                _, predicted = torch.max(outputs.data, 1)

                tp, fp, tn, fn = confusion(predicted, labels)

                true_positives += tp
                false_positives += fp
                true_negatives += tn
                false_negatives += fn

                total += labels.size(0)
                accuracy += (predicted == labels).sum().item()

            # compute the accuracy over all test segment
            accuracy = 100 * accuracy / total
            print(f"Got {accuracy} ")

            sensitivity = true_positives / (true_positives + false_negatives)
            specificity = true_negatives / (true_negatives + false_positives)
            precision = true_positives / (true_positives + false_positives)
            recall = true_positives / (true_positives + false_negatives)
            F1 = (2 * precision * recall) / (precision + recall)

            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"F1: {F1}")
            print(f"Youden's index: {sensitivity + specificity - 1 }")

            # Plot confusion matrix

            cm = [[true_positives, false_negatives], [false_positives, true_negatives]]

            ax = plt.subplot()
            sns.heatmap(cm, annot=True, fmt="g", cmap="Greens", ax=ax)
            # annot=True to annotate cells, ftm='g' to disable scientific notation

            # labels, title and ticks
            ax.set_xlabel("Predicted labels")
            ax.set_ylabel("True labels")
            # ax.set_title('Confusion Matrix');
            ax.xaxis.set_ticklabels(["apnea", "normal"])
            ax.yaxis.set_ticklabels(["apnea", "normal"])


check_accuracy("test", model)

Checking accuracy on test data
Got 64.44981862152358 


ZeroDivisionError: ignored